# Mapping Essential Ocean Variables to Usage Measurements

## intro

NSV is managing a number of crosswalks between the skos-concepts in the "AtlantOS Essential Variables" [(A05)](http://vocab.nerc.ac.uk/collection/P01/current/) collection and those in the "Parameter Usage" [(P01)](http://vocab.nerc.ac.uk/collection/P01/current/) collection.

These mappings allow to select all the relevant concrete Parameters on a detail level by using a very general conceptual top level reference to one of these so called 'EV' (essential variables) in oceanographic research.

Historically the first set of mappings were created using the "Parameter Usage Vocabulary ontology" ([puv](https://w3id.org/env/puv#)).
A more elaborate set of mappings (covering more EVs) was later added using the "i-ADOPT Framework ontology" ([iop](https://w3id.org/iadopt/ont/)).

At the time of writing the following EV are supported per strategy:
| strategy | supported ev code | ev label    | supported object of interest | object of interest label  |
|----------|-------------------|-------------|------------------------------|---------------------------|
| puv      | EV_OXY            | Oxygen      |                              |                           |
| puv      | EV_SALIN          | Salinity    |                              |                           |
| puv      | EV_SEATEMP        | Temperature |                              |                           |
| iop      | EV_OXY            | Oxygen      |                              |                           |
| iop      | EV_CO2            | Carbonate   |                              |                           |
| iop      | EV_SALIN          | Salinity    |                              |                           |
| iop      | EV_SEATEMP        | Temperature |                              |                           |
| iop      | EV_NUTS           | Nutrients   | CS026904                     | phosphate (PO43)          |
| iop      | EV_NUTS           | Nutrients   | CS026903                     | silicate (SiO44)          |
| iop      | EV_NUTS           | Nutrients   | CS026905                     | nitrate+nitrite (NO3-NO2) |
| iop      | EV_NUTS           | Nutrients   | CS002877                     | nitrite (NO2)             |
| iop      | EV_NUTS           | Nutrients   | CS002879                     | nitrate (NO3)             |

Below we show how to exploit and query both strategies.


At the heart of this are the separated sparql templates `./nsv-eov-to-usage_via-{strategy}.sparql` (where strategy in ['puv', 'iop'])i

These are organised to produce the exact same columns, based on the same required parameters.  

## basic python setup

In [4]:
from pykg2tbl import DefaultSparqlBuilder, KGSource, QueryResult
from pathlib import Path
from pandas import DataFrame


THIS_PATH = Path().absolute()

# SPARQL EndPoint to use - wrapped as Knowledge-Graph 'source'
NSV_ENDPOINT: str = "https://vocab.nerc.ac.uk/sparql/sparql"
NSV:KGSource = KGSource.build(NSV_ENDPOINT)

TEMPLATES_FOLDER = str(THIS_PATH)
GENERATOR = DefaultSparqlBuilder(templates_folder=TEMPLATES_FOLDER)

OUT_PATH = THIS_PATH / "results"
OUT_PATH.mkdir(exist_ok=True)

STRATEGIES = ['puv', 'iop']

def generate_sparql(name: str, **vars) -> str: 
    """ Simply build the sparql by using the named query and applying the vars
    """
    return GENERATOR.build_syntax(name, **vars)


def find_and_save_files(strategy: str, eov, obj= None, suffix: str = None) -> DataFrame:
    """ Finds the usage but also locally saves the output and generated files if a suffix is provided
    """
    assert strategy in STRATEGIES, f"***ERROR*** The only supported strategies are { STRATEGIES } "
    saving = bool(suffix)
    suffix = suffix or "any"
    fname_base = f"{ strategy }-{ eov }-{ suffix }"
    name = f"nsv-eov-to-usage_via-{strategy}.sparql"
    obj = obj if obj is None or isinstance(obj, list) else [obj]

    sparql = generate_sparql(name, eov=eov, obj=obj)
    if saving:
        fname_sparql: str = str(OUT_PATH / f"{ fname_base }.sparql")
        with open(fname_sparql, "w") as file:
            file.write(sparql)

    result: QueryResult = NSV.query(sparql=sparql)
    if saving:
        fname_csv: str = str(OUT_PATH / f"{ fname_base }.csv")
        result.as_csv(fname_csv)
    
    print(f"done for { fname_base } --> found { len(result) }")

    return result.to_dataframe()


def find_usage(strategy: str, eov, obj= None, *ignore) -> DataFrame:
    """ Finds the usage and returns it
    """
    return find_and_save_files(strategy, eov, obj, None)

## lookup the usage for a specific case

Checking up on 'Salinity' with the 'i-ADOPT' strategy



In [11]:
find_usage('iop', 'EV_SALIN')

done for iop-EV_SALIN-any --> found 33


,ConceptIRI,P01ID,P01Label,P09IDS,P02IDS,R03IDS
0,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::ODSDM021,Salinity of the water body,SDN:P09::SSAL,SDN:P02::PSAL,NaN
1,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALBRBR,Practical salinity of the water body by RBR MS...,NaN,SDN:P02::PSAL,NaN
2,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALBSTX,Practical salinity of the water body by bench ...,NaN,SDN:P02::PSAL,NaN
3,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALCC01,Practical salinity of the water body by CTD an...,NaN,SDN:P02::PSAL,NaN
4,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALCC02,Practical salinity of the water body by CTD (s...,NaN,SDN:P02::PSAL,NaN
5,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALCU01,Practical salinity of the water body by CTD an...,NaN,SDN:P02::PSAL,NaN
6,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALCU02,Practical salinity of the water body by CTD (s...,NaN,SDN:P02::PSAL,NaN
7,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALMC01,Practical salinity of the water body by moving...,NaN,SDN:P02::PSAL,NaN
8,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALMV01,Practical salinity of the water body by moving...,NaN,SDN:P02::PSAL,NaN
9,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::PSALPR01,Practical salinity of the water body by conduc...,NaN,SDN:P02::PSAL,NaN


## run over all cases to compare them


In [12]:
cases: list = [
    ('puv', 'EV_OXY', None, 'any'),
    ('puv', 'EV_SALIN', None, 'any'),
    ('puv', 'EV_SEATEMP', None, 'any'),
    ('iop', 'EV_OXY', None, 'any'),
    ('iop', 'EV_CO2', None, 'any'),
    ('iop', 'EV_SALIN', None, 'any'),
    ('iop', 'EV_SEATEMP', None, 'any'),
    ('iop', 'EV_NUTS', None, 'any'),
    ('iop', 'EV_NUTS', 'CS026904', 'PO43'),    # phosphate
    ('iop', 'EV_NUTS', 'CS026903', 'SiO44'),   # silicate
    ('iop', 'EV_NUTS', 'CS026905', 'NO2NO3'),  # nitrate+nitrite
    ('iop', 'EV_NUTS', 'CS002877', 'NO2'),     # nitrite
    ('iop', 'EV_NUTS', 'CS002879', 'NO3'),     # nitrate
    ('iop', 'EV_NUTS', ['CS002879', 'CS002877', 'CS026905'], 'NOx'),     # nitrate, nitrite, nitrate+nitrite
]
# run over the above cases as arguments to pass
# NOTE: this will locally create files *sparql and *csv one can then compare
for args in cases:
    find_and_save_files(*args)

done for puv-EV_OXY-any --> found 22
done for puv-EV_SALIN-any --> found 33
done for puv-EV_SEATEMP-any --> found 60
done for iop-EV_OXY-any --> found 22
done for iop-EV_CO2-any --> found 18
done for iop-EV_SALIN-any --> found 33
done for iop-EV_SEATEMP-any --> found 59
done for iop-EV_NUTS-any --> found 110
done for iop-EV_NUTS-PO43 --> found 23
done for iop-EV_NUTS-SiO44 --> found 19
done for iop-EV_NUTS-NO2NO3 --> found 25
done for iop-EV_NUTS-NO2 --> found 22
done for iop-EV_NUTS-NO3 --> found 21
done for iop-EV_NUTS-NOx --> found 68
